In [ ]:
!pip install keras_preprocessing

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 1.8 MB/s eta 0:00:00


In [ ]:
!pip install dill

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 3.4 MB/s eta 0:00:00


In [ ]:
import functools
import operator
import os
import time
import numpy as np
import dill

In [ ]:
%run '/content/drive/MyDrive/Colab Notebooks/config.ipynb'

0.005
400


In [ ]:
%run '/content/drive/MyDrive/Colab Notebooks/inference_model.ipynb'

0.005
400
50
400
/content/drive/MyDrive/N_model_lr5e-3
30


In [ ]:
class VideoDescriptionInference():
  """
  Initialize the parameters for the model
  """
  def __init__(self):
    self.latent_dim = latent_dim
    self.num_encoder_tokens = num_encoder_tokens
    self.num_decoder_tokens = num_decoder_tokens
    self.time_steps_encoder = time_steps_encoder
    self.max_probability = max_probability

    self.tokenizer, self.inf_encoder_model, self.inf_decoder_model = inference_model()
    self.save_model_path = save_model_path
    self.test_path = test_path
    self.search_type = search_type

  def greedy_search(self, loaded_array):
    """
    :param f: the loaded numpy array after creating videos to frames and extracting features
    :return : the final sentence which has been predicted greedily
    """
    inv_map = self.index_to_word()
    states_value = self.inf_encoder_model.predict(loaded_array.reshape(-1, 50, 4096))
    target_seq = np.zeros((1, 1, 400))
    final_sentence = ''
    target_seq[0, 0, self.tokenizer.word_index['bos']] = 1
    for i in range(15):
      output_tokens, h, c = self.inf_decoder_model.predict([target_seq] + states_value)
      states_value = [h, c]
      output_tokens = output_tokens.reshape(self.num_decoder_tokens)
      y_hat = np.argmax(output_tokens)
      if y_hat == 0:
        continue
      if inv_map[y_hat] is None:
        break
      if inv_map[y_hat] == 'eos':
        break
      else:
        final_sentence = final_sentence + inv_map[y_hat] + ' '
        target_seq = np.zeros((1, 1, 400))
        target_seq[0, 0, y_hat] = 1

    final_sentence = final_sentence.strip().capitalize()

    final_sentence = final_sentence.rstrip() + '.'

    return final_sentence

  def index_to_word(self):
    index_to_word = {value: key for key, value in self.tokenizer.word_index.items()}
    return index_to_word

  def get_test_data(self):
    """
    Loads all the numpy files for testing
    : return: Two lists containing all the vidoe arrays and the video Ids.
    """
    X_test = []
    X_test_filename = []

    Test_feature_dir = "/content/drive/MyDrive/test_vgg16feature_50"

    # List all the feature files in the 'test_feature' folder
    feature_files = [f for f in os.listdir(Test_feature_dir) if f.endswith('.npy')]
    print("Feature Files:", feature_files)

    if not feature_files:
      print("No feature files found in the directory.")
      return X_test, X_test_filename

    for feature_file in feature_files:
      # Load the feature file
      f = np.load(os.path.join(Test_feature_dir, feature_file))
      print("Loaded Feature Shape:", f.shape)

      # Extract the video ID from the feature filen name
      video_id = os.path.splitext(feature_file)[0]

      X_test.append(f)
      X_test_filename.append(video_id)

    X_test = np.array(X_test)
    return X_test, X_test_filename

  def test(self):
    """
    writes the captions of all the testing videos in a text file
    """
    print("Inside test function...")
    X_test, X_test_filename = self.get_test_data()
    print(X_test)
    print(X_test_filename)

    # generate inference test outputs
    # Opens a text file for writing the test results, and the file name is based on the specified search type (greedy or beam)
    with open(os.path.join('/content/drive/MyDrive', 'test50_%s.txt' % self.search_type), 'w') as file:
      for idx, x in enumerate(X_test):
        print("Processing video:", X_test_filename[idx])
        file.write(X_test_filename[idx] + ' ')

        print("Search Type:", self.search_type)

        if self.search_type == 'greedy':
          start = time.time()
          decoded_sentence = self.greedy_search(x.reshape(-1, 50, 4096))
          file.write(''.join(decoded_sentence))
        file.write('\n')

        # re-initialize max-probability
        self.max_probability = -1

In [ ]:
if __name__ == "__main__":
  video_to_text = VideoDescriptionInference()
  video_to_text.test()

Inside test function...
Feature Files: ['1.npy', '10.npy', '11.npy', '12.npy', '13.npy', '14.npy', '15.npy', '16.npy', '17.npy', '18.npy', '19.npy', '2.npy', '20.npy', '21.npy', '22.npy', '23.npy', '24.npy', '25.npy', '26.npy', '27.npy', '28.npy', '29.npy', '3.npy', '30.npy', '31.npy', '32.npy', '33.npy', '34.npy', '35.npy', '36.npy', '37.npy', '38.npy', '39.npy', '4.npy', '5.npy', '6.npy', '7.npy', '8.npy', '9.npy']
Loaded Feature Shape: (50, 4096)
Loaded Feature Shape: (50, 4096)
Loaded Feature Shape: (50, 4096)
Loaded Feature Shape: (50, 4096)
Loaded Feature Shape: (50, 4096)
Loaded Feature Shape: (50, 4096)
Loaded Feature Shape: (50, 4096)
Loaded Feature Shape: (50, 4096)
Loaded Feature Shape: (50, 4096)
Loaded Feature Shape: (50, 4096)
Loaded Feature Shape: (50, 4096)
Loaded Feature Shape: (50, 4096)
Loaded Feature Shape: (50, 4096)
Loaded Feature Shape: (50, 4096)
Loaded Feature Shape: (50, 4096)
Loaded Feature Shape: (50, 4096)
Loaded Feature Shape: (50, 4096)
Loaded Feature Sha